In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = "https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html"

In [ ]:
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [ ]:
states = soup.find_all('div', class_='g-state g-cat-all')

In [ ]:
#All States
State = []
Status = []
BusinessUpdate = []
MaskUpdate = []
URL = []

for state in states:
    try:
        name = state.find('div',class_='g-name').text.split('›')[0]
        info = state.find('span', class_="g-text").text
        business = state.find('span', class_='g-rule g-businesses').text
        masks = state.find('span', class_='g-rule g-masks').text
        links = state.find('span',class_='g-link')
        link = links.find('a', href=True)
        if (name):
            State.append(name)
            Status.append(info)
            BusinessUpdate.append(business)
            MaskUpdate.append(masks)
            URL.append(link.get('href'))
    except AttributeError as e:
        print(e)

In [ ]:
COVID_dict={"State":State,"Status":Status,"Business Update":BusinessUpdate,"Mask Update":MaskUpdate,"URL":URL}
COVID=pd.DataFrame(COVID_dict)
COVID

In [ ]:
#Change the Username
COVID.to_excel(r"C:\Users\bpappas\Downloads\US-COVID.xlsx",index=False)